In [91]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestRegressor
import random

# Database connection
DB_USER = "pamudithasenanayake"
DB_PASS = "123"
DB_HOST = "localhost"
DB_NAME = "fashionsite"

engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}")


In [92]:
def fetch_fashion_data_tool(limit=100):
    print("[DataAgent] Fetching fashion data from DB...")
    df = pd.read_sql(f"SELECT * FROM synthetic_fashion_trends ORDER BY timestamp DESC LIMIT {limit}", engine)
    df['hashtags'] = df['hashtags'].apply(lambda x: list(x) if x else [])
    df['tags'] = df['tags'].apply(lambda x: list(x) if x else [])
    df['trend_score'] = df['trend_score'] + np.random.uniform(-0.05, 0.05, size=len(df))
    df['trend_score'] = df['trend_score'].clip(0, 1)
    return df


In [93]:
class TrendPredictorAgent:
    def __init__(self):
        self.model = RandomForestRegressor(n_estimators=50, random_state=42)

    def train(self, df):
        print("[ScorePredictorAgent] Training trend predictor...")
        df['content_length'] = df['content'].str.len()
        df['num_hashtags'] = df['hashtags'].apply(len)
        df['num_tags'] = df['tags'].apply(len)
        features = ['likes', 'shares', 'comments', 'content_length', 'num_hashtags', 'num_tags']
        self.model.fit(df[features], df['trend_score'])
        return self


def predict_missing_scores_tool(df, predictor):
    print("[ScorePredictorAgent] Predicting missing trend scores...")
    mask = df['trend_score'].isnull()
    if mask.sum() > 0:
        df_missing = df[mask].copy()
        df_missing['content_length'] = df_missing['content'].str.len()
        df_missing['num_hashtags'] = df_missing['hashtags'].apply(len)
        df_missing['num_tags'] = df_missing['tags'].apply(len)
        features = ['likes', 'shares', 'comments', 'content_length', 'num_hashtags', 'num_tags']
        df.loc[mask, 'trend_score'] = predictor.model.predict(df_missing[features])
    df['predicted_trend_score'] = df['trend_score']
    return df


In [94]:
class ForecastAgent:
    def __init__(self):
        self.model = RandomForestRegressor(n_estimators=50, random_state=42)

    def prepare_features(self, df):
        df = df.sort_values(by=['trend_name', 'timestamp'])
        df['prev_score'] = df.groupby('trend_name')['trend_score'].shift(1)
        df['prev_score2'] = df.groupby('trend_name')['trend_score'].shift(2)
        df['prev_score3'] = df.groupby('trend_name')['trend_score'].shift(3)
        df['rolling_mean'] = df.groupby('trend_name')['trend_score'].transform(
            lambda x: x.rolling(3, min_periods=1).mean())
        df['rolling_std'] = df.groupby('trend_name')['trend_score'].transform(
            lambda x: x.rolling(3, min_periods=1).std().fillna(0))
        df = df.fillna(0)
        return df


def train_forecast_tool(df, agent):
    print("[ForecasterAgent] Training forecast model...")
    df = agent.prepare_features(df)
    features = ['prev_score', 'prev_score2', 'prev_score3', 'likes', 'shares', 'comments', 'rolling_mean',
                'rolling_std']
    agent.model.fit(df[features], df['trend_score'])
    return agent


def forecast_trends_tool(df, agent):
    print("[ForecasterAgent] Forecasting trends...")
    df = agent.prepare_features(df)
    features = ['prev_score', 'prev_score2', 'prev_score3', 'likes', 'shares', 'comments', 'rolling_mean',
                'rolling_std']
    df['forecasted_trend_score'] = agent.model.predict(df[features])
    return df[['trend_name', 'forecasted_trend_score']]


In [95]:
class TrendDirectionAgent:
    def __init__(self, up_threshold=0.01, down_threshold=-0.01):
        self.up_threshold = up_threshold
        self.down_threshold = down_threshold

    def compute_direction(self, df, score_column='trend_score'):
        print("[DirectionAgent] Computing trend directions...")
        df = df.sort_values(by=['trend_name', 'timestamp'])
        df['prev_score'] = df.groupby('trend_name')[score_column].shift(1)
        df['score_change'] = df[score_column] - df['prev_score']
        df['trend_direction'] = df['score_change'].apply(
            lambda x: 'up' if x > self.up_threshold else ('down' if x < self.down_threshold else 'stable'))
        df['trend_direction'] = df['trend_direction'].fillna('stable')
        return df


In [96]:
def compute_overall_direction_tool(df, predicted_col='predicted_trend_score', forecast_col='forecasted_trend_score',
                                   up_threshold=0.01, down_threshold=-0.01):
    print("[InsightsAgent] Aggregating overall trend directions...")
    agg = df.groupby('trend_name').agg({predicted_col: 'mean', forecast_col: 'mean'}).reset_index()
    agg['score_change'] = agg[forecast_col] - agg[predicted_col]
    agg['trend_direction'] = agg['score_change'].apply(
        lambda x: 'up' if x > up_threshold else ('down' if x < down_threshold else 'stable'))
    return agg[['trend_name', predicted_col, forecast_col, 'trend_direction']]


In [97]:
from agents import Runner, Agent, OpenAIChatCompletionsModel
from openai import AsyncOpenAI


class FashionTrendOrchestrator:
    def __init__(self, gemini_api_key: str):
        self.gemini_client = AsyncOpenAI(
            base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
            api_key=gemini_api_key
        )
        self.gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=self.gemini_client)
        instructions = (
            "You are the Fashion Trend Orchestrator Agent. "
            "You have multiple sub-agents (DataAgent, ScorePredictorAgent, ForecasterAgent, DirectionAgent, InsightsAgent). "
            "Decide which agent to call next and explain briefly before calling. "
            "Return the final aggregated insights and top forecasted trends."
        )
        self.agent = Agent(name="Gemini Fashion Orchestrator", instructions=instructions, model=self.gemini_model)

    def run_pipeline(self, content_summary: str) -> str:
        print("[OrchestratorAgent] Running multi-agent pipeline...")
        result = Runner.run(self.agent, input=content_summary)
        return result.output_text


In [98]:
# 1. Fetch data
df = fetch_fashion_data_tool(limit=100)

# 2. Train predictor and fill missing scores
predictor_agent = TrendPredictorAgent().train(df)
df = predict_missing_scores_tool(df, predictor_agent)

# 3. Train forecast agent and predict
forecast_agent = ForecastAgent()
forecast_agent = train_forecast_tool(df, forecast_agent)
df_forecast = forecast_trends_tool(df, forecast_agent)

# 4. Compute trend directions
direction_agent = TrendDirectionAgent()
df = direction_agent.compute_direction(df, score_column='predicted_trend_score')

# 5. Merge forecast & direction
df_final = df.merge(df_forecast, on='trend_name', how='left')
print(df_final[['trend_name', 'predicted_trend_score', 'forecasted_trend_score', 'trend_direction']].head(20))

# 6. Compute overall trend
df_overall = compute_overall_direction_tool(df_final)
print(df_overall.sort_values(by='forecasted_trend_score', ascending=False).head(20))


[DataAgent] Fetching fashion data from DB...
[ScorePredictorAgent] Training trend predictor...
[ScorePredictorAgent] Predicting missing trend scores...
[ForecasterAgent] Training forecast model...
[ForecasterAgent] Forecasting trends...
[DirectionAgent] Computing trend directions...
    trend_name  predicted_trend_score  forecasted_trend_score trend_direction
0   bucket hat               0.739293                0.711560          stable
1   bucket hat               0.739293                0.572816          stable
2   bucket hat               0.739293                0.634794          stable
3   bucket hat               0.739293                0.446027          stable
4   bucket hat               0.739293                0.735001          stable
5   bucket hat               0.739293                0.051348          stable
6   bucket hat               0.739293                0.122061          stable
7   bucket hat               0.516612                0.711560            down
8   bucket hat

/Users/pamudithasenanayake/Downloads/fashion_project/backend/.venv/lib/python3.12/site-packages/sqlalchemy/engine/cursor.py:1137: RuntimeWarning: coroutine 'FashionAI._analyze_async' was never awaited
  rows = dbapi_cursor.fetchall()
/var/folders/vf/rbbd31d52x71bpbg3sn62p3r0000gn/T/ipykernel_93748/4255584270.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
/var/folders/vf/rbbd31d52x71bpbg3sn62p3r0000gn/T/ipykernel_93748/4255584270.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
